# INSTALL: packages

In [ ]:
# no additional packages required

# EXECUTE: WalletClustering_AddressIdentification notebook

In [ ]:
%run ./WalletClustering_AddressIdentification.ipynb # includes list creation of addresses
# methods & variables of notebook can be referenced

In [2]:
# only to remember lists
blacklistAddresses = pickle.load(open('output/blacklistAddresses.pickle', 'rb')) # list of exchange wallet addresses

In [ ]:
# print(blacklistAddresses)

# DEFINE: update associations in neo4j

In [5]:
#checks if there is already an association in place. If yes, skip entry
check_association = """Match (a:Address {address: '%s'}) 
where a.association is not null
return true"""

In [6]:
def updateAssociations(inputDf):
    associations = list(inputDf['association'].unique())
    
    for item in associations:
       
        projection = inputDf[inputDf['association'].str.match(item)]
        addresses = list(projection["address"].values)
        response = conn.query(check_association % addresses[0],db='neo4j')
        if not response:
            query = """CALL apoc.periodic.iterate( 'UNWIND $addresses as item return item',
                        'Match (a:Address {address: item}) set a.association = "%s" return a', 
                        {batchSize:1000, parallel:true, iterateList:true, params:{addresses:%s}})""" % (item, str(addresses))
            result = conn.query(query,db='neo4j')
        else:
            print(addresses[0] + ": There is already an association in the DB")

In [ ]:
# def updateAssociations(inputDf):
#    query = """
#        MATCH (a:Address {address: "%s"})
#        set a.association = %s,
#        Return a.address
#        """
#
#    for index, row in inputDf.iterrows():
#        address = inputDf['address'][index]
#        association = inputDf['association'][index]
#        conn.query(query % (address, association), db='neo4j')


# EXECUTE: update associations in neo4j

In [8]:
updateAssociations(blacklistAddresses)

13642EkbFPxomz7pZ8eAdGXyTTXp9zRTb2: There is already an association in the DB
